<a href="https://colab.research.google.com/github/leonardgst/Time_Series_project/blob/main/Connection_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8Bdxr5fhkX2ke9M9ZWH68Favdit_jf8Z3z40WUMzrME&tc=MaA6I7cisvWzM5GARM0pM9L1FLDXtkHm49evEuRQx3w&cc=OpklAVyXK7F-up84fvikay9wFhvNpMVrvIvTBbOQ9FI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkkW6tALTfdnyXI8zrYadFsWD5nQcF21fJggMVVp1S1gLoHag3pY_g

Successfully saved authorization token.


In [3]:
ee.Initialize()